In [1]:
import json
import os

# Task PreTENS: Presupposed Taxonomies: Evaluating Neural Network Semantics)

- [Link Documentazione](http://www.italianlp.it/resources/semeval-2022-pretens-evaluating-neural-networks-on-presuppositional-semantic-knowledge/)

Il task è diviso in due sotto-task.

1) Classificazione di frasi come accettabili o non accettabili
2) Predizione (regressione) del punteggio medio di accettabilità che varia da 1 a 7, assegnato alle frasi da annotatori umani.

Il dataset è stato costruito per 3 lingue: inglese, francese ed italiano.

Cercheremo di risolvere il secondo sotto-task per l'italiano.

Tag-IT era un task di classificazione di documenti, questo è un task di **regressione** su **frasi**.

I file di training utilizzati per questo laboratorio sono:
- https://github.com/shammur/SemEval2022Task3/blob/main/data/train/train_subtask-2/it/It-Subtask2-fold_0.tsv
- https://github.com/shammur/SemEval2022Task3/blob/main/data/train/train_subtask-2/it/It-Subtask2-fold_1.tsv
    
Il file di test è:
- https://github.com/shammur/SemEval2022Task3/blob/main/data/test/official_test_set_with_labels/subtask-2/It-Subtask2-scores.tsv

# Unisco i due file di training

In [2]:
src_files = ['data/PreTENS/src/It-Subtask2-fold_0.tsv', 'data/PreTENS/src/It-Subtask2-fold_1.tsv']
out_path = 'data/PreTENS/src/It-Subtask2-train.tsv'

In [3]:
with open(out_path, 'w+') as out_file:
    for line in open(src_files[0], 'r'):
        out_file.write(line)
    first = True
    for line in open(src_files[1], 'r'):
        if not first:
            out_file.write(line)
        first = False

# Preparo l'input per Profiling-UD

Creo un file per ogni frase, usando come nome del file l'id della frase.

Creo a parte un file in cui ogni id è associato allo score da predire e allo split (train/test) in cui si trova la frase.

In [12]:
train_path = 'data/PreTENS/src/It-Subtask2-train.tsv'
test_path = 'data/PreTENS/src/It-Subtask2-scores.tsv'
profiling_input_dir = 'data/PreTENS/preprocessed_dataset/profiling_input'
sentences_info_path = 'data/PreTENS/preprocessed_dataset/sentences_info.json'

In [5]:
if not os.path.exists(profiling_input_dir):
    os.makedirs(profiling_input_dir)

In [6]:
sentences_info_dict = dict()

### Preprocessing del training set e del test set

Creazione di un file per ogni frase e un dizionario che contiene le info (score e split) di ogni frase

In [7]:
for line in open(train_path, 'r'):    
    splitted_line = line.strip().split('\t')
    if splitted_line[0] != 'ID':
        sent_id = f'train_{splitted_line[0]}'
        sentence = splitted_line[1]
        score = float(splitted_line[2])
                
        out_path = os.path.join(profiling_input_dir, f'{sent_id}.txt')
        with open(out_path, 'w+') as out_file:
            out_file.write(sentence)
    
        sentences_info_dict[sent_id] = {'score': score, 'split': 'train'}

In [8]:
for line in open(test_path, 'r'):    
    splitted_line = line.strip().split('\t')
    if splitted_line[0] != 'ID':
        sent_id = f'test_{splitted_line[0]}'
        sentence = splitted_line[2]
        score = float(splitted_line[3])
                
        out_path = os.path.join(profiling_input_dir, f'{sent_id}.txt')
        with open(out_path, 'w+') as out_file:
            out_file.write(sentence)
    
        sentences_info_dict[sent_id] = {'score': score, 'split': 'test'}

Creazione del file che contiene id della frase, score e split.

In [1]:
# sentences_info_dict

In [13]:
with open(sentences_info_path, 'w') as out_file:
    json.dump(sentences_info_dict, out_file)